# Partie 3 : Analyse exploratoire (2-3h)

## **Competence evaluee : C2.3 - Analyser des donnees structurees pour repondre a un besoin metier**

### Etape 3.1 : Statistiques descriptives

In [13]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Chemins
DATA_DIR = "../data"
OUTPUT_DIR = "../data/output"
PARQUET_DIR = "../data/output/consommations_clean"

In [3]:
# récupération des données précédentes
df_final = pd.read_csv("../data/output/consommations_enrichies.csv" , sep=",")

- Calculer les statistiques par type d'energie, type de batiment et commune

In [4]:
# Exemple de syntaxe
# Statistiques globales
stats = df_final.groupby(
    ['type_energie', 'type', 'commune']
).agg(
    consommation_totale=('consommation_clean', 'sum'),
    consommation_moyenne=('consommation_clean', 'mean'),
    cout_total=('cout_instantane', 'sum'),
    nb_enregistrements=('batiment_id', 'count')
).reset_index()

stats.head()


,type_energie,type,commune,consommation_totale,consommation_moyenne,cout_total,nb_enregistrements
0,eau,ecole,Bordeaux,115460.74,7.335498,418545.1825,15740
1,eau,ecole,Le Havre,129255.30,8.198357,468550.4625,15766
2,eau,ecole,Lyon,223744.22,7.042626,811072.7975,31770
3,eau,ecole,Marseille,228342.82,7.362572,827742.7225,31014
4,eau,ecole,Montpellier,40624.52,5.266337,147263.8850,7714


- Identifier les batiments les plus/moins energivores

In [5]:
# Top 5 consommations totales
batiments_top = df_final.groupby('batiment_id')['consommation_clean'].sum().sort_values(ascending=False).head(5)

# Bottom 5 consommations totales
batiments_bottom = df_final.groupby('batiment_id')['consommation_clean'].sum().sort_values(ascending=True).head(5)

# Optionnel : par m²
batiments_top_m2 = (df_final.groupby('batiment_id')
                    .apply(lambda x: x['consommation_clean'].sum() / x['surface_m2'].iloc[0])
                    .sort_values(ascending=False)
                    .head(5))

batiments_top_m2.head(5)


batiment_id
BAT0043    15182.606161
BAT0121    15077.982289
BAT0112    15055.991212
BAT0122    14916.248385
BAT0005    14772.305280
dtype: float64

- Calculer la repartition des consommations par classe energetique DPE

In [6]:
# Somme par classe énergétique
repartition_dpe = df_final.groupby('classe_energetique')['consommation_clean'].sum()

# Pourcentage
repartition_dpe_pct = repartition_dpe / repartition_dpe.sum() * 100
repartition_dpe_pct


classe_energetique
A     0.216207
B     2.400991
C     4.624103
D    13.428859
E    12.191502
F    33.213987
G    33.924352
Name: consommation_clean, dtype: float64

- Analyser l'evolution temporelle (tendances mensuelles, saisonnalite)

In [19]:
output_dir = "../data/output/graphics"
os.makedirs(output_dir, exist_ok=True)

# Consommation mensuelle moyenne par type d'énergie
consommation_mensuelle = df_final.groupby(['year','month','type_energie'])['consommation_clean'].mean().reset_index()

# Pour tracer
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,6))
sns.lineplot(data=consommation_mensuelle, x='month', y='consommation_clean', hue='type_energie', marker='o')
plt.title("Évolution mensuelle moyenne de la consommation par type d'énergie")

# sauvegarde de l'image dans data/output
plt.savefig("../data/output/graphics/06_consommation_mensuelle.png", dpi=300, bbox_inches='tight')
plt.close()


- Comparer la consommation theorique (selon DPE) vs reelle

In [ ]:
# Consommation théorique moyenne par type et DPE
theorique = df_final.groupby(['type', 'classe_energetique'])['moyenne_type'].mean().reset_index()

# Consommation réelle par type et DPE
reelle = df_final.groupby(['type', 'classe_energetique'])['consommation_clean'].mean().reset_index()

# Fusion pour comparaison
comparaison = reelle.merge(theorique, on=['type','classe_energetique'])
comparaison['ecart_pct'] = (comparaison['consommation_clean'] - comparaison['moyenne_type']) / comparaison['moyenne_type'] * 100
comparaison


,type,classe_energetique,consommation_clean,moyenne_type,ecart_pct
0,ecole,B,93.759993,0.084182,111277.993492
1,ecole,C,59.018500,0.084182,70008.390003
2,ecole,D,93.847462,0.084182,111381.898410
3,ecole,E,112.018226,0.084182,132967.045646
4,ecole,F,180.975610,0.084182,214881.887965
5,ecole,G,175.878806,0.084182,208827.368213
6,gymnase,C,113.056059,0.137677,82017.126422
7,gymnase,D,161.150244,0.137677,116949.851208
8,gymnase,E,178.770456,0.137677,129748.114628
9,gymnase,F,288.666846,0.137677,209570.247049
